In [2]:
!pip install streamlit
!pip install docling
!pip install pymupdf
!pip install sentence-transformers
!pip install langchain-core
!pip install langchain-community
!pip install langchain-text-splitters
!pip install langchain_community
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 89.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.7/202.7 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.6/86.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━

In [3]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from docling.datamodel.pipeline_options import PdfPipelineOptions, TableFormerMode
from docling.document_converter import PdfFormatOption, InputFormat
from docling.backend.docling_parse_v2_backend import DoclingParseV2DocumentBackend
from huggingface_hub import InferenceClient
from docling.document_converter import DocumentConverter
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re, fitz
import os
import tempfile
import streamlit as st
from dotenv import load_dotenv

load_dotenv()

# ======== Docling PDF Pipeline Options ========
pipeline_options = PdfPipelineOptions()
pipeline_options.do_ocr = False
pipeline_options.do_table_structure = True
pipeline_options.table_structure_options.mode = TableFormerMode.FAST

PDF_FORMAT_OPTIONS = {
    InputFormat.PDF: PdfFormatOption(
        pipeline_options=pipeline_options,
        backend=DoclingParseV2DocumentBackend
    )
}

# ======== Embedding Model ========
EMBEDDING_MODEL = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"  # ✅ miniLM model
)

# ======== HF Inference Client ========
# Read token from environment (recommended) or hardcode for testing
HF_TOKEN = os.getenv("HF_TOKEN", os.getenv("HFTOKEN"))
client = InferenceClient(token=HF_TOKEN)



def getMarkDown(pdf_path):
    doc_converter = DocumentConverter(format_options=PDF_FORMAT_OPTIONS)
    conv_result = doc_converter.convert(pdf_path)
    return conv_result.document.export_to_markdown()


def getHeadingsData(pdf_path):
    headings_dict = {}
    heading_types = {1: (24, float('inf')), 2: (15, 18)}

    doc = fitz.open(pdf_path)
    total_pages = len(doc)

    # Start from page 7 (index 6) and skip last two pages
    for page_num in range(6, total_pages - 2):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]

        for block in blocks:
            if "lines" not in block:
                continue
            for line in block["lines"]:
                for span in line["spans"]:
                    size = span["size"]
                    text = span["text"].strip()
                    is_bold = "bold" in span["font"].lower()

                    if len(text) <= 1 or text.isdigit() or not is_bold or size < 13:
                        continue

                    heading_type = None
                    for h_type, (min_size, max_size) in heading_types.items():
                        if min_size <= size <= max_size:
                            heading_type = h_type
                            break

                    if heading_type:
                        cleaned_text = re.sub(r"[',’]", "", text)
                        if cleaned_text not in headings_dict:
                            headings_dict[cleaned_text] = {
                                "level": heading_type,
                                "page_no": page_num + 1  # convert to 1-based
                            }

    doc.close()
    return headings_dict

def getLessonHeadings(headings):
    level1_headings = [(title, info['page_no']) for title, info in headings.items() if info['level'] == 1]
    level1_headings.sort(key=lambda x: x[1])

    ranges = {}
    for i, (title, start_page) in enumerate(level1_headings):
        end_page = level1_headings[i + 1][1] - 1 if i + 1 < len(level1_headings) else float('inf')
        ranges[title] = (start_page, end_page)
    return ranges


def preprocess_markdown(markdown_text, headings):
    md_lines = markdown_text.splitlines(keepends=True)
    fixed_lines = []

    for line in md_lines:
        match = re.match(r'^(#+)\s+(.*)', line)
        if match:
            md_level = len(match.group(1))
            md_text = match.group(2).strip()
            cleaned_md_text = re.sub(r"[',’]", "", md_text)

            if cleaned_md_text in headings:
                correct_level = headings[cleaned_md_text]["level"]
                fixed_lines.append('#' * correct_level + ' ' + md_text + '\n' if md_level != correct_level else line)
            else:
                fixed_lines.append('#' * 3 + ' ' + md_text + '\n')
        else:
            fixed_lines.append(line)

    return "".join(fixed_lines)


def split_and_add_metadata(output_content, headings, ranges):
    headers_to_split_on = [
        ("#", "Lesson"),
        ("##", "Heading"),
        ("###", "Sub Heading"),
    ]

    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on)
    md_header_splits = markdown_splitter.split_text(output_content)

    current_lesson = None
    for doc in md_header_splits:
        heading_text = doc.metadata.get("Heading")
        if heading_text:
            cleaned_heading = re.sub(r"[',’]", "", heading_text)
            if cleaned_heading in headings:
                page_no = headings[cleaned_heading]["page_no"]
                doc.metadata["page_no"] = page_no
                for lesson, (start_page, end_page) in ranges.items():
                    if start_page <= page_no <= end_page:
                        doc.metadata["Lesson"] = lesson
                        current_lesson = lesson
                        break
            elif current_lesson:
                doc.metadata.setdefault("Lesson", current_lesson)
        elif current_lesson:
            doc.metadata.setdefault("Lesson", current_lesson)

    recursive_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    final_chunks = recursive_splitter.split_documents(md_header_splits)

    for idx, doc in enumerate(final_chunks):
        header_title = doc.metadata.get("Heading", "")
        sub_heading = doc.metadata.get("Sub Heading", "")
        if sub_heading:
            doc.page_content = f"{header_title}\n{sub_heading}\n{doc.page_content}"
        else:
            doc.page_content = f"{header_title}\n{doc.page_content}"
        doc.metadata["index"] = idx

    return final_chunks


def embed(chunks):
    vectorstore = FAISS.from_documents(chunks, EMBEDDING_MODEL)
    return vectorstore


def pipeline(pdf_path, faiss_path=None):
    print("1️⃣ Extracting markdown...")
    markdown_text = getMarkDown(pdf_path)

    print("2️⃣ Extracting headings...")
    headings = getHeadingsData(pdf_path)

    print("3️⃣ Mapping lessons...")
    ranges = getLessonHeadings(headings)

    print("4️⃣ Preprocessing markdown...")
    cleaned_md = preprocess_markdown(markdown_text, headings)

    print("5️⃣ Splitting & adding metadata...")
    final_chunks = split_and_add_metadata(cleaned_md, headings, ranges)

    print("6️⃣ Embedding chunks...")
    retriever = embed(final_chunks)

    # Extract lesson names for dropdown in Streamlit
    lesson_names = list(ranges.keys())

    print("✅ Pipeline complete")
    return retriever, final_chunks, lesson_names
# ---------------------- Common Utilities ----------------------

def split_text_into_chunks(text, chunk_size=3000, chunk_overlap=200):
    """Split text into chunks of approximately `chunk_size` characters."""
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    return splitter.split_text(text)


def retrieve_lesson_content_by_metadata(all_docs, lesson_name):
    """
    Retrieve and combine all chunks for a specific lesson using metadata.
    """
    filtered_docs = [doc for doc in all_docs if doc.metadata.get("Lesson") == lesson_name]
    filtered_docs.sort(key=lambda doc: doc.metadata.get("index", 0))
    return "\n".join(doc.page_content for doc in filtered_docs)


def call_llama_chat(
    client,
    messages,
    model="meta-llama/Llama-3.1-8B-Instruct",
    max_tokens=1000,
    system_prompt="You are a helpful assistant that answers using the provided context."
):
    """
    Generic wrapper for llama chat completions with a system role always included.
    """
    # Always prepend system message
    messages = [{"role": "system", "content": system_prompt}] + messages

    response = client.chat_completion(
        model=model,
        messages=messages,
        max_tokens=max_tokens
    )
    return response["choices"][0]["message"]["content"].strip()

# ---------------------- Lesson Actions ----------------------

def summarize_lesson_text(long_text, client):
    """Generate a concise summary of a long lesson text."""
    chunks = split_text_into_chunks(long_text)

    # Summarize each chunk
    chunk_summaries = []
    for chunk in chunks:
        messages = [{"role": "user", "content": f"Summarize the following text briefly:\n\n{chunk}"}]
        summary = call_llama_chat(client, messages)
        chunk_summaries.append(summary)

    # Combine all summaries into a final one
    combined_summaries = "\n\n".join(chunk_summaries)
    messages = [{"role": "user", "content": f"Combine the following summaries into a concise final summary:\n\n{combined_summaries}"}]
    return call_llama_chat(client, messages)


def generate_questions_from_lesson(long_text, client):
    """Generate comprehension questions from a lesson."""
    chunks = split_text_into_chunks(long_text)

    chunk_questions = []
    for chunk in chunks:
        messages = [{"role": "user", "content": (
            "Based on the following text, generate a list of diverse questions "
            "(factual, analytical, inferential), numbered:\n\n" + chunk
        )}]
        questions = call_llama_chat(client, messages)
        chunk_questions.append(questions)

    combined_questions = "\n\n".join(chunk_questions)
    messages = [{"role": "user", "content": f"Merge the following questions into one coherent, non-repetitive list:\n\n{combined_questions}"}]
    return call_llama_chat(client, messages)
def rewrite_query(conversation_history, user_query, client):
    prompt = f"""You are a query rewriting model for a retrieval-augmented generation (RAG) system.
Your task is to analyze the conversation history and the user's latest query, then generate a new, optimized query.

---
Instructions:
1. If follow-up: Combine context from `Conversation History` with `User's Latest Query` into a standalone query.
2. If standalone: Improve clarity, add relevant keywords.
3. Output only the rewritten query.

---
Conversation History:
{conversation_history}

User's Latest Query:
{user_query}

Rewritten Query:"""

    messages = [{"role": "user", "content": prompt}]
    return call_llama_chat(client, messages)
def handle_user_query(user_query, client, vectorstore, conversation_history):
    # Step 1: Rewrite query
    optimized_query = rewrite_query(conversation_history, user_query, client)

    # Step 2: Retrieve docs
    retrieved_docs = vectorstore.similarity_search(optimized_query, k=6)
    context = "\n\n".join(doc.page_content for doc in retrieved_docs)

    # Step 3: Generate final answer
    messages = [{"role": "user", "content": f"Context:\n{context}\n\nUser Query:\n{user_query}"}]
    return call_llama_chat(client, messages)

def main():
    st.title("Smart Learning App")

    # Step 1: Select class & subject
    class_selected = st.selectbox("Select Class", ["Class 1","Class 2","Class 3","Class 4","Class 5","Class 6", "Class 7", "Class 8","Classs 9","Class 10","others"])
    subject_selected = st.selectbox("Select Subject", ["Math", "Science", "English"])
    #mode_selected = st.radio("Choose Mode:", ["Normal", "Language"])

    # Step 2: Upload PDF
    uploaded_file = st.file_uploader("Upload your textbook (PDF)", type=["pdf"])


    if uploaded_file is not None:
        st.success("PDF uploaded successfully!")

        # Run pipeline only once and store results in session_state
        if "pipeline_done" not in st.session_state:
            with st.spinner("Processing PDF..."):
                with tempfile.NamedTemporaryFile(delete=False,suffix=".pdf") as tmp_file:
                    tmp_file.write(uploaded_file.getbuffer())
                    tmp_pdf_path=tmp_file.name


                   # retriever, final_chunks, lesson_names = language_pipeline(tmp_pdf_path)
                retriever, final_chunks, lesson_names = pipeline(tmp_pdf_path)
                st.session_state.retriever = retriever
                st.session_state.final_chunks = final_chunks
                st.session_state.lesson_names = lesson_names
                st.session_state.pipeline_done = True

        # Step 3: Lesson Selectionss
        lesson_selected = st.selectbox("Select Lesson", st.session_state.lesson_names)

        # Step 4: Task Selection
        task = st.radio("Choose an option:", ["Summarise", "Question Forming", "Doubt Clearing"])

        # Retrieve lesson content
        lesson_text = retrieve_lesson_content_by_metadata(st.session_state.final_chunks, lesson_selected)

        # ------------------ One-shot Tasks ------------------
        if task == "Summarise":
            if st.button("Summarise Lesson"):
                with st.spinner("Generating summary..."):
                    summary = summarize_lesson_text(lesson_text, client)
                st.subheader("📌 Lesson Summary")
                st.write(summary)

        elif task == "Question Forming":
            if st.button("Generate Questions"):
                with st.spinner("Generating questions..."):
                    questions = generate_questions_from_lesson(lesson_text, client)
                st.subheader("❓ Comprehension Questions")
                st.write(questions)

        # ------------------ Doubt Clearing (Conversational) ------------------
        elif task == "Doubt Clearing":
            if "chat_history" not in st.session_state:
                st.session_state.chat_history = []

            user_query = st.text_input("Ask your doubt:")

            if st.button("Submit Doubt"):
                if user_query.strip():
                    # Append user message
                    st.session_state.chat_history.append({"role": "user", "content": user_query})

                    # Get answer via RAG
                    answer = handle_user_query(
                        user_query,
                        client,
                        st.session_state.retriever,
                        st.session_state.chat_history
                    )
                    st.session_state.chat_history.append({"role": "assistant", "content": answer})

            # Display chat
            for chat in st.session_state.chat_history:
                role = "🧑 You" if chat["role"] == "user" else "🤖 Assistant"
                st.markdown(f"**{role}:** {chat['content']}")


if __name__ == "__main__":
    main()

/tmp/ipython-input-903350370.py:32: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  EMBEDDING_MODEL = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2025-08-22 05:18:31.973 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-22 05:18:32.219 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-08-22 05:18:32.220 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-22 05:18:32.222 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-22 05:18:32.224 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-22 05:18:32.226 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-22 05:18:32.227 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-22 05:18:32.229 Thread 'MainThread': mi